In [2]:
# Read me: 
"""
This Notebook uses 'yellow_tripdata_2013-05.csv' dataset which 
represent real taxi calls over manhetten recorded for may 2013
You can download it here: https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-05.csv

How to use : 
1) Download: the csv from the link above.
2) Location: The file must be in the same directory as this notebook.
3) Run: Run all cell consecutively.


Returned values:
1) "clean_2013.csv" : all valid trips from 0000 hours Sunday, May 5,2013, to 2359 hours, Saturday May 11, 2013.
    This used to create "Request nodes" in the project , and containes 4 columns:
    
    a)   request_Index: Natural .
    b)   pickup_datetime : pd.Timestamp .
    c&d) src and dst: both : Natural .
    
    !!!! IMPORTANT !!!!
   (src and dst node ID's are matching this graph: 
    G = ox.graph_from_place('Manhattan, New York City, New York, USA', network_type='drive'))
    
2) "all_paths_nyc.pickle" : Dictionary, keyed by source and target, of shortest paths.
    display the path as a list of node ID's.


3) "all_travel_times.pickle ": (source, dictionary) iterator with dictionary keyed by target and shortest path length as the key value.

How to use : 
1) Download: the csv from the link above.
2) Location: The file must be in the same directory as this notebook.
3) Run: Run all cell consecutively.


Enjoy (=
"""


'\nThis Notebook uses \'yellow_tripdata_2013-05.csv\' dataset which \nrepresent real taxi calls over manhetten recorded for may 2013\nYou can download it here: https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-05.csv\n\nHow to use : \n1) Download: the csv from the link above.\n2) Location: The file must be in the same directory as this notebook.\n3) Run: Run all cell consecutively.\n\n\nReturned values:\n1) "clean_2013.csv" : all valid trips from 0000 hours Sunday, May 5,2013, to 2359 hours, Saturday May 11, 2013.\n    This used to create "Request nodes" in the project , and containes 4 columns:\n    \n    a)   request_Index: Natural .\n    b)   pickup_datetime : pd.Timestamp .\n    c&d) src and dst: both : Natural .\n    \n    !!!! IMPORTANT !!!!\n   (src and dst node ID\'s are matching this graph: \n    G = ox.graph_from_place(\'Manhattan, New York City, New York, USA\', network_type=\'drive\'))\n    \n2) "all_paths_nyc.pickle" : Dictionary, keyed by source and target, 

In [3]:
import pandas as pd
import collections
import networkx as nx
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
ox.config(use_cache=True, log_console=True)
ox.__version__
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import pickle


In [4]:
# Making request from real nyc taxi data , for Simulation

In [5]:
# Getting the Data
df2013 = pd.read_csv('yellow_tripdata_2013-05.csv')
# Sorting by pickupo time
df2013 = df2013.sort_values(by=['pickup_datetime'])
print(df2013.head())

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


         vendor_id      pickup_datetime     dropoff_datetime  passenger_count  \
10661523       VTS  2013-05-01 00:00:00  2013-05-01 00:12:00                1   
10638632       VTS  2013-05-01 00:00:00  2013-05-01 00:24:00                1   
10634665       VTS  2013-05-01 00:00:00  2013-05-01 00:07:00                1   
10634664       VTS  2013-05-01 00:00:00  2013-05-01 00:08:00                1   
10643988       VTS  2013-05-01 00:00:00  2013-05-01 00:27:00                3   

          trip_distance  pickup_longitude  pickup_latitude  rate_code  \
10661523           2.61        -73.986757        40.742767          1   
10638632          12.90        -73.988747        40.736955          1   
10634665           2.60        -73.991260        40.745135          1   
10634664           1.29        -74.000580        40.742137          1   
10643988           6.48        -73.983583        40.761282          1   

         store_and_fwd_flag  dropoff_longitude  dropoff_latitude payment_t

In [6]:
# Parser func
def str_to_time(s):
    t = pd.Timestamp(s)
    tt = t.to_pydatetime()
    return tt
    
def str_to_time_list(s):
    time_list = []
    for t_string in s :
        time_list.append(str_to_time(t_string))
    return time_list

In [7]:
# Parse pickup_datetime string to pd.Timestamp()
df2013['pickup_datetime'] =str_to_time_list(df2013['pickup_datetime'])
df2013['dropoff_datetime'] =str_to_time_list(df2013['dropoff_datetime']) 

In [8]:
# Filter only the requests from 0000 hours Sunday, May 5,2013, to 2359 hours, Saturday May 11, 2013,
df2013 = df2013[df2013.pickup_datetime > pd.Timestamp(2013, 5, 5, 0)  ]
df2013 = df2013[df2013.dropoff_datetime < pd.Timestamp(2013, 5, 11, 23,59) ]


In [9]:
# Drops rows with bad data (bad ccordinates)

df2013 = df2013[df2013.pickup_longitude != 0]
df2013 = df2013[df2013.pickup_latitude != 0]
df2013 = df2013[df2013.dropoff_longitude != 0]
df2013 = df2013[df2013.dropoff_latitude != 0]
df2013 = df2013[df2013.passenger_count != 0]

df2013.dropna()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
8265950,CMT,2013-05-05 00:00:01,2013-05-05 00:16:39,1,3.3,-73.969635,40.752034,1,N,-74.003017,40.733278,CRD,14.0,0.5,0.5,3.0,0.0,18.0
8940596,CMT,2013-05-05 00:00:01,2013-05-05 00:02:55,2,0.4,-73.979877,40.726943,1,N,-73.982349,40.731810,CSH,4.0,0.5,0.5,0.0,0.0,5.0
8265322,CMT,2013-05-05 00:00:01,2013-05-05 00:18:06,1,2.9,-73.951133,40.710833,1,N,-73.980719,40.729852,CRD,14.5,0.5,0.5,3.1,0.0,18.6
8927973,CMT,2013-05-05 00:00:01,2013-05-05 00:08:53,1,1.3,-73.978998,40.767025,1,N,-73.988513,40.758998,CSH,7.5,0.5,0.5,0.0,0.0,8.5
8946795,CMT,2013-05-05 00:00:01,2013-05-05 00:02:27,1,0.5,-73.985484,40.741528,1,N,-73.981604,40.747176,CSH,4.0,0.5,0.5,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749640,CMT,2013-05-11 23:56:16,2013-05-11 23:57:46,1,0.2,-73.979624,40.746354,1,N,-73.983221,40.747901,CSH,3.0,0.5,0.5,0.0,0.0,4.0
11758559,CMT,2013-05-11 23:56:21,2013-05-11 23:57:33,2,0.5,-73.963026,40.775122,1,N,-73.957686,40.782174,CSH,3.5,0.5,0.5,0.0,0.0,4.5
9834676,CMT,2013-05-11 23:56:26,2013-05-11 23:58:00,2,0.8,-73.965043,40.761847,1,N,-73.956668,40.767064,CRD,5.0,0.5,0.5,1.0,0.0,7.0
9845923,CMT,2013-05-11 23:56:46,2013-05-11 23:58:57,1,2.8,-73.977482,40.754201,4,N,-73.970842,40.752742,CRD,3.5,0.5,0.5,2.0,0.0,6.5


In [10]:
#Drop out of bound pick up and dropoff
df2013 = df2013[df2013.pickup_longitude > -74.022 ]
df2013 = df2013[df2013.pickup_latitude > 40.69]
df2013 = df2013[df2013.dropoff_longitude > -74.022]
df2013 = df2013[df2013.dropoff_latitude > 40.69]
df2013 = df2013[df2013.pickup_longitude < -73.906 ]
df2013 = df2013[df2013.pickup_latitude < 40.88]
df2013 = df2013[df2013.dropoff_longitude < -73.906]
df2013 = df2013[df2013.dropoff_latitude < 40.88]


In [11]:
# Drop unwanted columns
df2013=df2013.drop(columns=['vendor_id','trip_distance','rate_code','store_and_fwd_flag','payment_type','fare_amount','surcharge','mta_tax','tip_amount','tolls_amount','total_amount','passenger_count'])

In [ ]:
df2013.head()

In [ ]:
df2013.tail()

In [13]:
# New York map for processing
G = ox.graph_from_place('Manhattan, New York City, New York, USA', network_type='drive')
print(type(G))
# fig, ax = ox.plot_graph(G, node_zorder=2, node_color='w', bgcolor='k')


<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [12]:
# impute missing edge speeds then calculate edge travel times
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [ ]:
ox.stats.basic_stats(G)

In [ ]:
if 42436396 in G.nodes():
    print('nigger')
else:
    print('peller')

In [14]:
# Adding df2013['src'] & df2013['dst']
# get the nearest network nodes to two points
df2013['src'] = ox.distance.nearest_nodes(G, df2013['pickup_longitude'], df2013['pickup_latitude']) 
df2013['dst'] = ox.distance.nearest_nodes(G, df2013['dropoff_longitude'], df2013['dropoff_latitude'])

In [ ]:
df2013.head()

In [15]:
# Drop unwanted columns
df2013=df2013.drop(columns=['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','dropoff_datetime'])

In [16]:
# Checking if every src node and every dst node are indeed in our graph G
validity = True
graph_nodes = G.nodes()
for s_id in df2013['src']:
    if s_id not in graph_nodes:
        validity = Flase  
    
for s_id in df2013['dst']:
    if s_id not in graph_nodes:
        validity = Flase
if validity:
    print('src and dst are valid')
else:
    print('There is a bad src or dst')

src and dst are valid


In [17]:
# Saving the df as csv
df2013.to_csv('clean_2013.csv' , index=True )

In [22]:
if 1775176474 in G.nodes():
    print("yeah")
else:
    print("nah")

yeah


In [20]:
temp = nx.single_source_dijkstra_path(G, 42440754, weight='travel_times')


In [21]:
print(temp)

{42440754: [42440754], 5145323793: [42440754, 5145323793], 42439001: [42440754, 42439001], 42431482: [42440754, 42431482], 42436957: [42440754, 42436957], 42439006: [42440754, 42439001, 42439006], 42436947: [42440754, 42439001, 42436947], 589100791: [42440754, 42431482, 589100791], 42437830: [42440754, 42431482, 42437830], 3624790934: [42440754, 42436957, 3624790934], 42446571: [42440754, 42436957, 42446571], 42440743: [42440754, 42439001, 42439006, 42440743], 42436944: [42440754, 42439001, 42439006, 42436944], 42436949: [42440754, 42439001, 42436947, 42436949], 42446562: [42440754, 42439001, 42436947, 42446562], 4161568888: [42440754, 42431482, 42437830, 4161568888], 5145330172: [42440754, 42431482, 42437830, 5145330172], 370896902: [42440754, 42431482, 42437830, 370896902], 1989931095: [42440754, 42436957, 3624790934, 1989931095], 42446584: [42440754, 42436957, 3624790934, 42446584], 42455013: [42440754, 42436957, 42446571, 42455013], 42431470: [42440754, 42439001, 42439006, 42440743

In [24]:
temp2 = nx.single_source_dijkstra_path(G, 42440754, weight='travel_times')

In [25]:
print(temp2)

{42440754: [42440754], 5145323793: [42440754, 5145323793], 42439001: [42440754, 42439001], 42431482: [42440754, 42431482], 42436957: [42440754, 42436957], 42439006: [42440754, 42439001, 42439006], 42436947: [42440754, 42439001, 42436947], 589100791: [42440754, 42431482, 589100791], 42437830: [42440754, 42431482, 42437830], 3624790934: [42440754, 42436957, 3624790934], 42446571: [42440754, 42436957, 42446571], 42440743: [42440754, 42439001, 42439006, 42440743], 42436944: [42440754, 42439001, 42439006, 42436944], 42436949: [42440754, 42439001, 42436947, 42436949], 42446562: [42440754, 42439001, 42436947, 42446562], 4161568888: [42440754, 42431482, 42437830, 4161568888], 5145330172: [42440754, 42431482, 42437830, 5145330172], 370896902: [42440754, 42431482, 42437830, 370896902], 1989931095: [42440754, 42436957, 3624790934, 1989931095], 42446584: [42440754, 42436957, 3624790934, 42446584], 42455013: [42440754, 42436957, 42446571, 42455013], 42431470: [42440754, 42439001, 42439006, 42440743

In [28]:
temp3 = nx.shortest_path(G, 1775176474, 42440754)

In [29]:
print(temp3)

[1775176474, 6176483595, 1775201764, 6177439749, 42436531, 42456049, 42427968, 42427970, 4886250352, 406006393, 5426969134, 370924677, 5779545445, 370912817, 42423774, 42426865, 7137281959, 42426875, 589100228, 42448714, 42451018, 42439191, 5137978709, 4161568888, 370896902, 42437830, 42431482, 42440754]


In [30]:
temp4 = nx.shortest_path(G, 42442469, 7684225787)

NetworkXNoPath: No path between 42442469 and 7684225787.

In [ ]:
# Shortest paths map generator

In [ ]:

#Testing if shortest paths match
#route = nx.shortest_path(G, 1773121034, 42445413, 'travel_time')
#print(route)
#route_map = ox.plot_route_folium(G, route)

In [ ]:
#travel_times = dict(nx.all_pairs_bellman_ford_path_length(G,weight='travel_times'))

In [ ]:
#path = dict(nx.all_pairs_dijkstra_path(G),weight='travel_times')


In [ ]:
#print(path[1773121034][42445413])
#print(type(path))

In [ ]:
#Saving the path map
#with open('all_paths_nyc.pickle ', 'wb') as handle:
  #  pickle.dump(path, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
#Saving the travel_times map
#with open('all_travel_times.pickle ', 'wb') as handle:
   # pickle.dump(travel_times, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
#with open('all_paths_nyc.pickle ', 'rb') as handle:
   # b = pickle.load(handle)
